# Introduction

![image](./imgs/‎GenAIEnterprises.‎004.png)

![image](./imgs/‎GenAIEnterprises.‎005.png)

![image](./imgs/‎GenAIEnterprises.‎006.png)

![image](./imgs/‎GenAIEnterprises.‎007.png)

![image](./imgs/‎GenAIEnterprises.‎008.png)



# Initial Setup

In [15]:
import boto3
import os
from IPython.display import display, Markdown
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings

def get_api_key(ssm_client, parameter_path):
    '''Get the OpenAI API key from the SSM Parameter Store'''
    try:
        response = ssm_client.get_parameter(
            Name=parameter_path,
            WithDecryption=True
        )
        return response['Parameter']['Value']
    except ssm_client.exceptions.ParameterNotFound:
        raise Exception(f'Parameter {parameter_path} not found in SSM Parameter Store')

# Create an SSM client using Boto3
region_name = os.getenv('AWS_REGION', 'us-east-1') 
ssm = boto3.client('ssm', region_name=region_name)

openai_api_key = get_api_key(ssm_client=ssm, parameter_path='/openai/api_key')
langchain_api_key = get_api_key(ssm_client=ssm, parameter_path='/langchain/api_key')


os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.langchain.plus"
os.environ['OPENAI_API_KEY'] = openai_api_key
os.environ["LANGCHAIN_API_KEY"] = langchain_api_key

# Set the model variable based on the current date
llm_model = "gpt-3.5-turbo-16k"

# Create the vector store and embedding function
embedding = OpenAIEmbeddings()
vectordb = Chroma(
    persist_directory='docs/chroma/',
    embedding_function=embedding
)

In [16]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

llm = ChatOpenAI(model_name=llm_model, temperature=0)
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(search_type="mmr", search_kwargs={'fetch_k': 5, 'k': 7}),
    return_source_documents=True,
)

# RFP Questions

In [ ]:
import pandas as pd

data = [{
  "category": "Access Control",
  "question": "What are the access policies at the company for repositories, task management systems, and other software used to provide services to prevent unauthorized access?"
},
{
  "category": "Data Security", 
  "question": "How does the company securely store and deploy results and software per industry best practices?"
},
{
  "category": "Data Security",
  "question": "What is the process at the company for returning or destroying a Customer's confidential information upon termination of an agreement?"
}, 
{
  "category": "Subcontractors",
  "question": "Is the company willing to remain fully responsible for the security of any subcontractors it uses?"
},
{
  "category": "Subcontractors",
  "question": "Will the company ensure any subcontractors it engages acknowledge the Customer's rights to modify results produced?"
},
{
  "category": "Intellectual Property",
  "question": "Does the company agree not to transfer, assign, license, pledge or encumber the results or related IP without explicit consent from the Customer?"
},
{
  "category": "Confidentiality",
  "question": "Does the company agree to hold a Customer's confidential information in confidence and not disclose it without permission?"
},
{
  "category": "Confidentiality",
  "question": "Does the company agree to return or destroy a Customer's confidential information upon termination of an agreement?"
},
{
  "category": "Confidentiality",
  "question": "Is the company willing to limit disclosures of a Customer's confidential information to legal process and to establish rights under an agreement?" 
},
{
  "category": "Warranties",
  "question": "Does the company warrant it will not violate rights of third parties?"
},
{
  "category": "Warranties",
  "question": "Does the company warrant it will not disclose or use a Customer's property if the Customer fails to pay?"
},
{
  "category": "Liability",
  "question": "Is the company willing to limit liability to the amount paid under the relevant Statement of Work?"
}]

df = pd.DataFrame(data)



In [ ]:
df['answer'] = ""

for index, row in df.iterrows():
    result = qa_chain({"query": row['question']})
    df.at[index, 'answer'] = result['result']

In [ ]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
import langchain
langchain.debug = False

df['score'] = ""

for index, row in df.iterrows():
    # prompt to evaluate how good company fit to the criteria in the question
    prompt = f"""Score the answer to the following question on a scale of 1-10.
    With the following criteria:
    0. question not answered
    1. company meets around 10% of the criteria
    2. company meets around 20% of the criteria
    3. company meets around 30% of the criteria
    4. company meets around 40% of the criteria
    5. company meets around 50% of the criteria
    6. company meets around 60% of the criteria
    7. company meets around 70% of the criteria
    8. company meets around 80% of the criteria
    9. company meets around 90% of the criteria
    10. company meets 100% of the criteria
    
    # Question:
    {row['question']}

    # Answer:
    {row['answer']}

    # Score:
    """
    result = qa_chain({"query": prompt})
    df.at[index, 'score'] = result['result']


# Export the results to a XLSX file

In [ ]:
df.style.to_excel('report.xlsx')

In [ ]:
df.style.background_gradient(axis=0)
